In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from plotting_utils import read_off
from toolbox.rescale import rescale

In [3]:
path_to_data = "../../data/meshes/"
figures = ["bull.off", "moomoo_s0.off"]
fig_data = [read_off(path_to_data + f) for f in figures]
rescaled_data = [(rescale(v, 0.03, 0.97), f) for v, f in fig_data]

In [4]:
from mesh_utils import vectorized_is_point_inside_mesh

N = 100
t = np.linspace(0, 1, N)
space = space = np.stack(np.meshgrid(t, t, t), axis=-1).reshape(N**3, -1)
fig_point_coords = [vectorized_is_point_inside_mesh(space, v, f) for v, f in rescaled_data]
fig_distrib=[a.reshape(N, N, N).astype(np.float32) for a in fig_point_coords]

/Users/acram/Desktop/VSCode_projects/advanced_ml/Convolutionnal-Wasserstein-Distances/notebooks/augustin/mesh_utils.py:23: RuntimeWarning: invalid value encountered in add
  mask_v = (v < 0.0) | ((u + v) > 1.0)


In [5]:
q = 5
t = np.linspace(0,1,q)
W = np.array([[s,1-s] for s in t])

In [6]:
from toolbox.blur_kernel import imgaussian
from toolbox.apply_3d_func import apply_3d_func
mu = N/25
blur = lambda x: imgaussian(x, mu, mu*50)
Kv = lambda x: apply_3d_func(blur, x)

In [72]:
def convolutional_barycenter(mus, alphas, area_weights, kernel, kernel_transpose, entropy_limit):
    n_iter = 1500
    tol = 1e-4
    v = np.ones(mus.shape)
    alphas = alphas / sum(alphas)
    if area_weights is None:
        area_weights = np.ones(mus.shape[1])
    if kernel_transpose is None:
        kernel_transpose = kernel
    barycenter = np.ones(mus.shape[1])
    for i in range(n_iter):
        old_barycenter = barycenter
        w = mus / (kernel_transpose(v * area_weights))
        d = v * kernel(w * area_weights)
        d[d<1e-100] = 1e-100
        barycenter = np.exp(np.sum(alphas.reshape(-1, 1) * np.log(d), axis=0))
        #entropy = -np.sum(area_weights*(barycenter*np.log(barycenter)))
        v = v*barycenter/d
        change = np.sum(np.abs(old_barycenter-barycenter) * area_weights)
        if i > 2 and change < tol :
            return barycenter
    return barycenter

In [75]:
w = W[1]
w = w/sum(w)
entropy_limit = None
Hv = np.array([dist.flatten()/dist.sum() for dist in fig_distrib])
B = convolutional_barycenter(Hv, w, None, Kv, None,entropy_limit)
B = B.reshape(N, N, N)
B = B/max(B.flatten())

(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
999999.4292140526
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.2394676755424034
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.09958547757198781
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.04392725710501118
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.02797693777216882
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.020606970801502606
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.016156485932132276
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.013149195244818727
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.011009388248469185
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.009399233472216264
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.008136183081870887
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.007124334981781259
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.006309624339108545
(2, 1000000)
(1000000,)
(2, 1000000)
(2, 1000000)
0.005642689242863291
(2, 1000000)
(

In [32]:
a = np.array([1, 2])
b = np.ones((2, 10))

In [42]:
a.reshape((2, 1)) * b

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]])